In [1]:
import json
from tqdm import tqdm
from utils import (
    calc_iou, any_match,
    gen_froc_plot, gen_tabular_data,
    gen_precision_recall_plot,
    convert_to_tensor
)
import pandas as pd
from sklearn.metrics import (
    precision_recall_curve,
    average_precision_score
)
from monai.data.box_utils import non_max_suppression
import torch

In [2]:
files = [
    'pixel-parsed-mixed_test-overjoyed-hen-305',
    'pixel-parsed-mixed_test-indecisive-eel-172',
    'pixel-parsed-mixed_test-crawling-sloth-537',
    'pixel-parsed-mixed_test-powerful-goat-766',
    'pixel-parsed-all'
]

In [3]:
all_fps_per_image = []
all_total_sensitivity = []
tabular_data = []
all_testy = []
all_lr_probs = []

composed_fp_probs = []
composed_tp_probs = []
composed_tp, composed_fp, composed_fn = 0, 0, 0
composed_testy = []
composed_lr_probs = []

for file_name in files:
    data_list_file_path = f'/data/output/validation/each_window/{file_name}.json'
    f = open(data_list_file_path)
    data = json.load(f)

    iou_thresh = 0.5 # same as Lucas
    num_images = 155
    num_targets = 313

    fp_probs = []
    tp_probs = []
    tp, fp, fn = 0, 0, 0

    testy = []
    lr_probs = []

    for content in tqdm(data['test']):

        # convert pred_box to monai standard arrangement mode
        converted = convert_to_tensor(content['pred_box'])

        keep = non_max_suppression(converted, torch.tensor(content['score']), nms_thresh=0.1, max_proposals=100)

        # boxes_after_nms = content['pred_box'][keep]
        boxes_after_nms = [content['pred_box'][index] for index in keep]
        scores_after_nms = torch.tensor(content['score'])[keep]

        for pred_box, score in zip(boxes_after_nms, scores_after_nms):
            # if score > 0.5:
            iou = 0
            if any_match(content['true_box'][0], pred_box):
                iou = calc_iou(content['true_box'][0], pred_box)
            if iou >= iou_thresh:
                tp += 1
                tp_probs.append(score)
                composed_tp += 1
                composed_tp_probs.append(score)

                testy.append(1)
                lr_probs.append(score)
                composed_testy.append(1)
                composed_lr_probs.append(score)
            if iou < iou_thresh and iou > 0:
                fp += 1
                fp_probs.append(score)
                composed_fp += 1
                composed_fp_probs.append(score)

                testy.append(0)
                lr_probs.append(score)
                composed_testy.append(0)
                composed_lr_probs.append(score)
            if iou == 0:
                fn += 1
                composed_fn += 1

                testy.append(0)
                lr_probs.append(score)
                composed_testy.append(0)
                composed_lr_probs.append(score)

    fps_per_image, total_sensitivity, precision, recall, p1, p2, p3, p4, p5, p6 ,p7, cpm = gen_tabular_data(
        tp, fp, fn, fp_probs, tp_probs, num_images, num_targets
    )

    all_fps_per_image.append(fps_per_image)
    all_total_sensitivity.append(total_sensitivity)
    all_testy.append(testy)
    all_lr_probs.append(lr_probs)

    tabular_data.append({
        "precision": precision,
        "recall": recall,
        "tp": tp,
        "fn": fn,
        "fp": fp,
        "1/8": p1,
        "2/8": p2,
        "4/8": p3,
        "1": p4,
        "2": p5,
        "4": p6,
        "8": p7,
        "CPM": cpm,
        "AP": str(average_precision_score(testy, lr_probs))
    })

first_table = pd.DataFrame(tabular_data)

  0%|          | 0/246 [00:00<?, ?it/s]


NameError: name 'convert_to_tensor' is not defined

In [ ]:
gen_froc_plot(all_fps_per_image, all_total_sensitivity)

In [ ]:
first_table

In [ ]:
gen_precision_recall_plot(precision_recall_curve, all_testy, all_lr_probs)